## Prediction

In [64]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import json
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [65]:
# Load the test dataset
test_dataset = load_dataset('json', data_files='./data/test_TLQA.json')['train']

In [66]:
def preprocess_function(examples):
    inputs = examples['question']
    targets = ["; ".join(ans) for ans in examples['answers']]  # Join list of answers with a separator
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    model_inputs['question'] = inputs  # Keep the original question
    return model_inputs

In [67]:
# Load the model and tokenizer
tokenizer = BartTokenizer.from_pretrained('./results-bart-base')
model = BartForConditionalGeneration.from_pretrained('./results-bart-base')

# Ensure the model is in evaluation mode
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [68]:
# Tokenize the test dataset
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [69]:
# Load the retriever model
retriever_model_name = "facebook/contriever"
retriever_tokenizer = AutoTokenizer.from_pretrained(retriever_model_name)
retriever_model = AutoModel.from_pretrained(retriever_model_name)

In [70]:


# Function to get context from Wikidata based on the type
def get_wikidata_context(wikidata_id, type_id):
    sparql_url = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?item ?itemLabel ?relatedItem ?relatedItemLabel WHERE {{
      wd:{wikidata_id} wdt:{type_id} ?relatedItem .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    headers = {'Accept': 'application/json'}
    response = requests.get(sparql_url, params={'query': query}, headers=headers)

    # Debugging: Print the response
    # print(f"Wikidata ID: {wikidata_id}")
    # print(f"Response: {response.text}")

    data = response.json()

    if 'results' in data and 'bindings' in data['results'] and data['results']['bindings']:
        related_items = []
        for binding in data['results']['bindings']:
            if 'relatedItemLabel' in binding:
                related_items.append(binding['relatedItemLabel']['value'])
                # print(f"Related item: {binding['relatedItemLabel']['value']}")
        return ", ".join(related_items)
    return ''



In [71]:
# Function to embed text using the retriever model
def embed_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [72]:
# Function to retrieve context using the retriever model
def retrieve_context(question, contexts, top_k=1):
    question_embedding = embed_text(question, retriever_tokenizer, retriever_model)
    context_embeddings = [embed_text(context, retriever_tokenizer, retriever_model) for context in contexts]
    similarities = cosine_similarity(question_embedding.detach().numpy(), np.vstack([ce.detach().numpy() for ce in context_embeddings]))
    top_k_indices = similarities[0].argsort()[-top_k:][::-1]
    return [contexts[i] for i in top_k_indices]

In [73]:
# Function to generate answers with context using the generative model
def generate_answers_with_context(question, context):
    combined_input = question + " " + context
    inputs = tokenizer(combined_input, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    outputs = model.generate(
        inputs['input_ids'],
        max_length=512,
        num_beams=5,
        early_stopping=True,
        repetition_penalty=2.5,
        length_penalty=1.0,
        no_repeat_ngram_size=2,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [74]:
# Inspect the dataset structure
print(test_dataset.column_names)
print(test_dataset[0])

['aliases', 'question', 'verified', 'subject', 'answers', 'subject_label', 'final_answers', 'wikidata_ID', 'type', 'answer_ids']
{'aliases': ['Anthony Paul Shaun Andrew Daure Grant'], 'question': 'List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.', 'verified': True, 'subject': 'Anthony Grant', 'answers': ['Southend United F.C. (2010, 2011, 2012)', 'Stevenage F.C. (2012, 2013)', 'Crewe Alexandra F.C. (2013, 2014, 2015)', 'Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)'], 'subject_label': 'Anthony Grant', 'final_answers': ['Southend United F.C. (2010, 2011, 2012)', 'Stevenage F.C. (2012, 2013)', 'Crewe Alexandra F.C. (2013, 2014, 2015)', 'Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)'], 'wikidata_ID': 'Q328656', 'type': 'P54', 'answer_ids': {'1. FC Kaiserslautern': None, '50PLUS': None, 'A C Wharton': None, 'A.A. Argentinos Juniors': None, 'A.E.K. Athens F.C.': None, 'A.S. Roma': None, 'AC Omonia': None, 'AC

In [75]:
# Generate predictions with context
predictions = []
for example in test_dataset:
    question = example['question']
    wikidata_id = example['wikidata_ID'] 
    type_id = example['type']  
    context = get_wikidata_context(wikidata_id, type_id)
    
    relevant_context = retrieve_context(question, [context], top_k=1)
    print(f"Context: {relevant_context}")
    prediction = generate_answers_with_context(question, " ".join(relevant_context))
    predictions.append(prediction)


Context: ['Stevenage F.C., Chelsea F.C., Luton Town F.C., Crewe Alexandra F.C., Oldham Athletic A.F.C., Port Vale F.C., Southend United F.C., Wycombe Wanderers F.C.']
Context: ["President of Ukraine, Chairman of the Verkhovna Rada, Prime Minister of Ukraine, People's Deputy of Ukraine, First Deputy Prime Minister of Ukraine, Head of the Security Service of Ukraine, Secretary of National Security and Defense Council of Ukraine"]
Context: ['Socialist Party, RISE (Ireland)']
Context: ['Amiens SC']
Context: ['Andrew Furey']
Context: ['University of Melbourne, University of Sydney, University of Toyama, Universiti Teknologi MARA, King Saud University']
Context: ['Taoiseach, deputy mayor, Teachta Dála, Tánaiste, Leader of Fine Gael, Minister for Defence, Irish Minister for Health, Minister for Jobs, Enterprise and Innovation, Minister for Social Protection, Minister for Transport, Tourism and Sport']
Context: ['Sergey Yaromko']
Context: ['Newport County A.F.C., Cardiff City F.C., Crystal Pal

In [76]:
# Save predictions to a JSON file
with open('predictions-Bart-base-RAG.json', 'w') as f:
    json.dump(predictions, f)

## Evaluation

In [77]:
from datasets import load_dataset
import json

In [78]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import re

# Define metric functions
def entity_match(predictions, references):
    scores = []
    for pred_list, ref_list in zip(predictions, references):
        pred_entities = set([re.sub(r'\s*\(.*?\)', '', pred.strip().lower()) for pred in pred_list])
        ref_entities = set([re.sub(r'\s*\(.*?\)', '', ref.strip().lower()) for ref in ref_list])

        # Debug prints
        # print("Pred Entities:", pred_entities)
        # print("Ref Entities:", ref_entities)

        matches = len(pred_entities & ref_entities)  # Intersection of sets
        scores.append(matches / len(ref_entities) if ref_entities else 0)
    return sum(scores) / len(scores)


def timeline_match(predictions, references):
    def extract_years(text):
        matches = re.findall(r'\d{4}', text)
        if matches:
            return set(map(int, matches))
        return set()

    scores = []
    for pred_list, ref_list in zip(predictions, references):
        match_scores = []
        for i, ref in enumerate(ref_list):
            ref_years = extract_years(ref)
            try:
                pred_years = extract_years(pred_list[i])
                # print(f"Matching prediction timeline {i + 1} with reference timeline {i + 1}")
                # print("Pred Years:", pred_years)
                # print("Ref Years:", ref_years)
                matches = len(pred_years & ref_years)
                match_scores.append(matches / len(ref_years) if ref_years else 0)
            except IndexError:
                # print(f"No prediction timeline for reference timeline {i + 1}")
                match_scores.append(0)
        scores.append(sum(match_scores) / len(match_scores) if match_scores else 0)
    return sum(scores) / len(scores)

def f1_metric(predictions, references):
    def compute_f1(pred_list, ref_list):
        pred_tokens = set(pred_list)
        ref_tokens = set(ref_list)

        if not ref_tokens:
            return 0.0

        true_positives = len(pred_tokens & ref_tokens)
        precision = true_positives / len(pred_tokens) if pred_tokens else 0
        recall = true_positives / len(ref_tokens) if ref_tokens else 0

        if precision + recall == 0:
            return 0.0

        return 2 * (precision * recall) / (precision + recall)

    f1_scores = []

    for pred_list, ref_list in zip(predictions, references):
        # Flatten the lists for token-level comparison
        all_preds = [token for pred in pred_list for token in pred.split()]
        all_refs = [token for ref in ref_list for token in ref.split()]
        f1_scores.append(compute_f1(all_preds, all_refs))

    return sum(f1_scores) / len(f1_scores) if f1_scores else 0

def extract_years(text):
    matches = re.findall(r'\d{4}', text)
    if matches:
        years = list(map(int, matches))
        return min(years), max(years)  # Return the earliest and latest years
    return None, None

def time_metric(predictions, references):
    time_diffs = []
    
    for pred_list, ref_list in zip(predictions, references):
        for pred, ref in zip(pred_list, ref_list):
            pred_start, pred_end = extract_years(pred)
            ref_start, ref_end = extract_years(ref)
            
            # print(f"Prediction: {pred}, Extracted Years: ({pred_start}, {pred_end})")  # Debugging
            # print(f"Reference: {ref}, Extracted Years: ({ref_start}, {ref_end})")  # Debugging
            
            if pred_start is not None and ref_start is not None:
                time_diff = abs(pred_start - ref_start) + abs(pred_end - ref_end)
                time_diffs.append(time_diff)
            else:
                time_diffs.append(float('inf'))  # Invalid prediction or reference

    valid_diffs = [diff for diff in time_diffs if diff != float('inf')]
    if not valid_diffs:
        return float('inf')  # If all are invalid, return inf
    return sum(valid_diffs) / len(valid_diffs) # Average time difference

def completeness(predictions, references):
    scores = []
    
    for pred_list, ref_list in zip(predictions, references):
        pred_items = set([item.strip().lower() for pred in pred_list for item in pred.split(", ")])
        ref_items = set([item.strip().lower() for ref in ref_list for item in ref.split(", ")])
        
        correct_count = len(pred_items.intersection(ref_items))
        total_count = len(ref_items)
        
        scores.append(correct_count / total_count if total_count > 0 else 0)
    return sum(scores) / len(scores)

In [79]:
# Load the test dataset
test_dataset = load_dataset('json', data_files='./data/test_TLQA.json')['train']

In [80]:
# References
references = [ans for ans in test_dataset['answers']]  # Ensure references are lists of answers

In [84]:
# Convert predictions to the required list format
predictions = [pred.split("; ") for pred in predictions]

In [82]:
# Load predictions from a JSON file
with open('predictions-Bart-base-RAG.json', 'r') as f:
    predictions = json.load(f)

In [85]:
# Debug: Print some predictions and references to ensure alignment
for pred, ref in zip(predictions, references):
    print("Prediction:", pred)
    print("Reference:", ref)

Prediction: ['Stevenage F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)', 'England national under-21 football team (2011)', 'Norwich City A.F., West Bromwich Albion FK (2012)']
Reference: ['Southend United F.C. (2010, 2011, 2012)', 'Stevenage F.C. (2012, 2013)', 'Crewe Alexandra F.C. (2013, 2014, 2015)', 'Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)']
Prediction: ['Deputy Prime Minister of Ukraine (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)', 'Chairman of the Ukrainian Parliament (2014)']
Reference: ['Prime Minister of Ukraine (2010)', 'First Deputy Prime Minister of Ukraine (2010)', "People's Deputy of Ukraine (2012, 2013, 2014, 2015)", 'Chairman of the Verkhovna Rada (2014, 2015, 2016, 2017, 2018, 2019, 2020)', 'President of Ukraine (2014)']
Prediction: ['Union for a Popular Movement (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018)', 'The Republicans (2018, 2019, 2020)']
Reference: ['Socialist Party (2010, 2011, 2012, 2013, 

In [86]:
# Evaluate predictions with custom metrics
entity_score = entity_match(predictions, references)
timeline_score = timeline_match(predictions, references)
f1 = f1_metric(predictions, references)
time_metric_score = time_metric(predictions, references)
completeness_score = completeness(predictions, references)

print(f"Entity Match: {entity_score}")
print(f"Timeline Match: {timeline_score}")
print(f"F1 Score: {f1}")
print(f"Time Metric: {time_metric_score}")
print(f"Completeness: {completeness_score}")

Entity Match: 0.1382000948527558
Timeline Match: 0.5598971883711612
F1 Score: 0.5415323180942757
Time Metric: 4.420526793823797
Completeness: 0.5590632677701333
